In [893]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sts
from sklearn.datasets import make_regression

In [894]:
X_, y_, coef = make_regression(n_samples=100, n_features=4, coef=True, bias=35, noise=30)
coef

array([10.50516744,  4.10952221, 41.35163437, 92.42591004])

In [905]:
class LR_econ_model(object):
    """Класс линейной регрессии для эконометрических моделей
    с проверками на значимость, интервалом предсказания, графиечксим представлением."""
    def __init__(self, X, y, standart = False):
        if standart: # стандартизация переменных
            self.regressors = (X - X.mean(axis = 0)) / X.std(axis = 0)
            self.target_train = (y - y.mean()) / y.std()
        else:
        #регрессоры
            self.regressors = X
        #вектор целевой переменной
            self.target_train = y
        #матрица регрессоров
        self.reg_corr_matrix = np.corrcoef(np.array([np.hsplit(X,
                                                            indices_or_sections = X.shape[1])]).reshape(X.shape[1], -1))
        if X.shape[1] > 1:
            # вектор корреляций регрессоров с целевой переменной
            self.C = np.corrcoef(x=np.array([np.hsplit(X,
                            indices_or_sections = X.shape[1])]).reshape(X.shape[1], -1),
                            y = self.target_train)[:-1,-1] 
            # линейный множественный коэффициент корреляции
            self.ryX = self.C.T @ np.linalg.inv(self.reg_corr_matrix) @ self.C
        
        
    def scatter_field(self):
        """График поля корреляции"""
        try:
            fig, ax = plt.subplots(figsize = (10, 7))
            ax.scatter(self.regressors, self.target_train , color = 'r') 
        except ValueError:
            print("You are trying to make a plot for more than one regressor!")
            ax.set_xlabel(xlabel = "x")
            ax.set_ylabel(ylabel = "y")
            ax.set_title(label = 'График поля корреляции')
            ax.grid()
            plt.show()
        
    
    def fit(self, plot = False):
        """Обучение модели"""
        I = np.ones(shape=(self.regressors.shape[0], 1)) # единичный вектор-столбец
        X = np.hstack((I, self.regressors))
        b = np.linalg.inv(np.dot(X.T, X)) @ X.T @ self.target_train # Аналитическое решение
        # формат вывода: (b_0, b_1,...b_n)
        if plot:
            try:
                fig, ax = plt.subplots(figsize = (10, 7))
                y_hat = b[0] + b[1] * self.regressors
                ax.scatter(self.regressors, self.target_train, color = 'r')
            except ValueError:
                print("You are trying to make a plot for more than one regressor!")
            else:
                ax.plot(self.regressors, y_hat, label = 'Прогноз')
                ax.set_xlabel(xlabel = "X")
                ax.set_ylabel(ylabel = "Y")
                ax.set_title(label = 'График МНК-оценки')
                ax.grid()
                plt.show()
        self.coefs = b
        return b
    
    def eq_significance(self, alpha = 0.05):
        """Значимость уравнения в целом"""
        I = np.ones(shape=(self.regressors.shape[0], 1)) # единичный вектор-столбец
        X = np.hstack((I, self.regressors))
        y_hat = self.coefs @ X.T
        # факторная вариация
        Q_r = ((y_hat - self.target_train.mean())**2).sum()
        # остаточная вариация
        Q_e = ((y_hat - self.target_train)**2).sum()
        S_r_squared = Q_r / self.regressors.shape[1] # факторная дисперсия
        S_e_squared = Q_e / (self.regressors.shape[0] - self.regressors.shape[1] - 1)
        F = S_r_squared / S_e_squared # расчетное значение F-критерия Фишера-Снедекора
        F_alpha = sts.f(self.regressors.shape[1], self.regressors.shape[0] - self.regressors.shape[1] - 1).isf(alpha)
        if F > F_alpha:
            print(f"{F} > {F_alpha}, следовательно уравнение значимо в целом")
        else:
            print(f"{F} < {F_alpha}, следовательно уравнение не значимо в целом")
        self.Q_r = Q_r
        self.Q_e = Q_e
        self.S_r_squared = S_r_squared
        self.S_e_squared = S_e_squared
        return Q_r, Q_e, S_r_squared, S_e_squared, F, F_alpha
    
    
    def coef_significance(self, alpha = 0.05):
        """Значимость параметров"""
        I = np.ones(shape=(self.regressors.shape[0], 1)) # единичный вектор-столбец
        X = np.hstack((I, self.regressors))
        Mat = np.linalg.inv(X.T @ X)
        t_alpha = sts.t(self.regressors.shape[0] - self.regressors.shape[1] - 1).isf(alpha)
        for j in range(X.shape[1]):
            m_j = np.sqrt(self.S_e_squared * Mat[j][j])
            t_stat_j = self.coefs[j] / m_j
            if abs(t_stat_j) > t_alpha:
                print(f"Так как |t_stat_{j}| = {abs(t_stat_j)} > t_alpha = {t_alpha}, коэффициент b_{j} значим на уровне значимости { alpha}")
            else:
                print(f"Так как |t_stat_{j}| = {abs(t_stat_j)} <= t_alpha = {t_alpha}, коэффициент b_{j} не значим на уровне значимости { alpha} и равен нулю")
            print(m_j, Mat[j][j])
            print("\n")
            
    def score(self):
        """Показатели качества"""
        # коэффициент детерминации
        R_squared = self.Q_r / (self.Q_e + self.Q_r)
        # скорретированный коэффициент детерминации
        R_squared_corrected = (self.regressors.shape[0] - self.regressors.shape[1] - 1) / (self.regressors.shape[0] - 1) * R_squared
        I = np.ones(shape=(self.regressors.shape[0], 1)) # единичный вектор-столбец
        X = np.hstack((I, self.regressors))
        # предсказания на тренировочной выборке
        y_hat = self.coefs @ X.T
        # средняя ошибка аппроксимации
        A = abs((self.target_train - y_hat)/self.target_train).sum() / (self.regressors.shape[0]) 
        print(f" R^2 = {R_squared}\n R^2_cor = {R_squared_corrected}\n A = {A * 100}%")
        
    def predict(self, X_predict, alpha = 0.05):
        """Предсказание"""
        I = np.ones(shape=(self.regressors.shape[0], 1)) # единичный вектор-столбец
        X = np.hstack((I, self.regressors))
        y_p = np.dot(self.coefs, X_predict)
        S_y_p = np.sqrt(1 + X_predict @ np.linalg.inv(X.T @ X) @ X_predict.T)
        t_alpha = sts.t(self.regressors.shape[0] - self.regressors.shape[1] - 1).isf(alpha)
        print(f"Точечный прогноз = {y_p}\n {alpha * 100}%-Доверительный интервал: ({y_p - S_y_p * t_alpha}, {y_p + S_y_p * t_alpha})")
        
        

In [916]:
X_test = np.array([[30, 4.1],
            [28, 3.1],
             [34, 3.05],
             [38, 3.2],
             [39, 3.4],
             [45, 3.05],
             [54, 3.7]])
y_test = np.array([23, 27, 31, 33, 34, 36, 37])
y = np.array([13, 16.5, 17, 15, 14.2, 10.5, 23, 12, 15.6, 12.5, 11.3, 13, 21, 12, 11, 11, 22.5, 26, 18.5, 13.2, 25.8, 17, 18, 21, 14.5, 23, 19.5, 14.2, 13.3, 16.1])
X1 = np.array([37, 60, 60, 53, 35, 30, 43, 30, 35, 32, 31, 33, 53, 32, 31, 36, 48, 55.5, 48, 44.1, 80, 60, 50, 54.6, 43, 66, 53.5, 45, 45, 50.6])
X2 = np.array([20, 10, 10, 15, 8, 15, 5, 10, 3, 5, 10, 5, 5, 20, 15, 5, 15, 10, 10, 25, 10, 12, 15, 20, 10, 5, 15, 12, 5, 10])
X = []
for i in range(len(X1)):
    X.append([X1[i], X2[i]])
X = np.array(X); X

array([[37. , 20. ],
       [60. , 10. ],
       [60. , 10. ],
       [53. , 15. ],
       [35. ,  8. ],
       [30. , 15. ],
       [43. ,  5. ],
       [30. , 10. ],
       [35. ,  3. ],
       [32. ,  5. ],
       [31. , 10. ],
       [33. ,  5. ],
       [53. ,  5. ],
       [32. , 20. ],
       [31. , 15. ],
       [36. ,  5. ],
       [48. , 15. ],
       [55.5, 10. ],
       [48. , 10. ],
       [44.1, 25. ],
       [80. , 10. ],
       [60. , 12. ],
       [50. , 15. ],
       [54.6, 20. ],
       [43. , 10. ],
       [66. ,  5. ],
       [53.5, 15. ],
       [45. , 12. ],
       [45. ,  5. ],
       [50.6, 10. ]])

In [917]:
obj = LR_econ_model(X_test, y_test, standart=1)

In [918]:
obj.regressors

array([[-0.99498448,  1.97843909],
       [-1.23515315, -0.73706554],
       [-0.51464714, -0.87284077],
       [-0.03430981, -0.46551508],
       [ 0.08577452,  0.07758585],
       [ 0.80628053, -0.87284077],
       [ 1.88703953,  0.89223724]])

In [919]:
obj.reg_corr_matrix

array([[1.        , 0.05623381],
       [0.05623381, 1.        ]])

In [920]:
obj.ryX

0.9519073237397925

In [921]:
obj.fit()

array([ 5.55111512e-17,  8.97351421e-01, -4.36743918e-01])

In [922]:
obj.eq_significance()

39.586373550494294 > 6.944271909999155, следовательно уравнение значимо в целом


(6.663351266178543,
 0.33664873382145616,
 3.3316756330892714,
 0.08416218345536404,
 39.586373550494294,
 6.944271909999155)

In [923]:
obj.coef_significance()

Так как |t_stat_0| = 5.062566688133451e-16 <= t_alpha = 2.1318467813362907, коэффициент b_0 не значим на уровне значимости 0.05 и равен нулю
0.10965021233473288 0.14285714285714285


Так как |t_stat_1| = 8.170813879622587 > t_alpha = 2.1318467813362907, коэффициент b_1 значим на уровне значимости 0.05
0.1098239948358454 0.1433103247386707


Так как |t_stat_2| = 3.9767622563260177 > t_alpha = 2.1318467813362907, коэффициент b_2 значим на уровне значимости 0.05
0.1098239948358454 0.1433103247386707




In [924]:
obj.score()

 R^2 = 0.9519073237397919
 R^2_cor = 0.6346048824931946
 A = 31.75136419623653%
